In [13]:
import numpy as np
import pandas as pd
import csv
import json
from sklearn import preprocessing

In [14]:
'''
ID - an Id that represents a (Shop, Item) tuple within the test set
shop_id - unique identifier of a shop
item_id - unique identifier of a product
item_category_id - unique identifier of item category
item_cnt_day - number of products sold. You are predicting a monthly amount of this measure
item_price - current price of an item
date - date in format dd/mm/yyyy
date_block_num - a consecutive month number, used for convenience. January 2013 is 0, February 2013 is 1,..., October 2015 is 33
item_name - name of item
shop_name - name of shop
item_category_name - name of item category
'''

'\nID - an Id that represents a (Shop, Item) tuple within the test set\nshop_id - unique identifier of a shop\nitem_id - unique identifier of a product\nitem_category_id - unique identifier of item category\nitem_cnt_day - number of products sold. You are predicting a monthly amount of this measure\nitem_price - current price of an item\ndate - date in format dd/mm/yyyy\ndate_block_num - a consecutive month number, used for convenience. January 2013 is 0, February 2013 is 1,..., October 2015 is 33\nitem_name - name of item\nshop_name - name of shop\nitem_category_name - name of item category\n'

In [15]:
items = pd.read_csv("../data/items.csv")
item_cat = pd.read_csv("../data/item_categories.csv")
train = pd.read_csv("../data/sales_train.csv")
sample_submission = pd.read_csv("../data/sample_submission.csv")
shops = pd.read_csv("../data/shops.csv")
test = pd.read_csv("../data/test.csv")

In [16]:
print('items : ', len(items))
print('item_cat : ', len(item_cat))
print('train : ' , len(train))
print('test : ', len(test))
print('shops : ', len(shops))

items :  22170
item_cat :  84
train :  2935849
test :  214200
shops :  60


In [18]:
print(min(train['item_cnt_day']), max(train['item_cnt_day']))

-22.0 2169.0


In [19]:
items.head()

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


In [20]:
item_cat.head()

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


In [21]:
train.head(10)
# train.tail()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0
5,10.01.2013,0,25,2564,349.00,1.0
6,02.01.2013,0,25,2565,549.00,1.0
7,04.01.2013,0,25,2572,239.00,1.0
8,11.01.2013,0,25,2572,299.00,1.0
9,03.01.2013,0,25,2573,299.00,3.0


In [22]:
train['date'] = train['date'].str[3:]

In [23]:
train

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,01.2013,0,59,22154,999.00,1.0
1,01.2013,0,25,2552,899.00,1.0
2,01.2013,0,25,2552,899.00,-1.0
3,01.2013,0,25,2554,1709.05,1.0
4,01.2013,0,25,2555,1099.00,1.0
...,...,...,...,...,...,...
2935844,10.2015,33,25,7409,299.00,1.0
2935845,10.2015,33,25,7460,299.00,1.0
2935846,10.2015,33,25,7459,349.00,1.0
2935847,10.2015,33,25,7440,299.00,1.0


In [29]:
train_groupby = train.groupby(['shop_id', 'item_id', 'date'])['item_cnt_day'].agg('sum').reset_index()
train_groupby

,shop_id,item_id,date,item_cnt_day
0,0,30,02.2013,31.0
1,0,31,02.2013,11.0
2,0,32,01.2013,6.0
3,0,32,02.2013,10.0
4,0,33,01.2013,3.0
...,...,...,...,...
1609119,59,22164,04.2015,2.0
1609120,59,22164,07.2015,1.0
1609121,59,22167,06.2014,1.0
1609122,59,22167,10.2013,1.0


In [28]:
#train_groupby_item[train_groupby_item['item_id'] == 2552]
train_groupby_item[train_groupby['item_id'] == 2552]

NameError: name 'train_groupby_item' is not defined

In [30]:
train_groupby_shop = train_groupby.groupby(['shop_id'])['item_cnt_day'].agg('sum').reset_index()
train_groupby_item = train_groupby.groupby(['item_id'])['item_cnt_day'].agg('sum').reset_index()
train_groupby_shop_and_item = train_groupby.groupby(['shop_id', 'item_id','date'])['item_cnt_day'].agg('sum').reset_index()

In [31]:
sample_submission.head()

,ID,item_cnt_month
0,0,0.5
1,1,0.5
2,2,0.5
3,3,0.5
4,4,0.5


In [32]:
shops.head()

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4


In [33]:
test.head() ##언젠지 없냐?

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [34]:
min(items['item_category_id'])

0

In [35]:
train_groupby_shop.head()

,shop_id,item_cnt_day
0,0,11705.0
1,1,6311.0
2,2,30620.0
3,3,28355.0
4,4,43942.0


In [36]:
train_groupby_shop = train_groupby_shop.rename(columns = {'item_cnt_day': '#2_amount'}, inplace = False)
train = pd.merge(train, train_groupby_shop, on='shop_id')

In [37]:
train['tmp'] = train['item_price']*train['item_cnt_day']
train_groupby_shop_price = train.groupby(['shop_id'])['tmp'].agg('sum').reset_index()
train_groupby_shop_price = train_groupby_shop_price.rename(columns = {'tmp': '#2_price'}, inplace = False)
train = pd.merge(train, train_groupby_shop_price, on ='shop_id')

In [38]:
train = pd.merge(train, items[['item_id', 'item_category_id']], on = 'item_id')

In [39]:
train_groupby_item_category = train.groupby(['item_category_id'])['item_cnt_day'].agg('sum').reset_index()

In [40]:
train = pd.merge(train, train_groupby_item_category , on='item_category_id')

In [41]:
train = train.rename(columns = {'item_cnt_day_y': '#3_amount'}, inplace = False)

In [42]:
train

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day_x,#2_amount,tmp,#2_price,item_category_id,#3_amount
0,01.2013,0,59,22154,999.0,1.0,48993.0,999.0,4.522658e+07,37,203284.0
1,01.2013,0,25,22154,999.0,1.0,241920.0,999.0,2.164806e+08,37,203284.0
2,01.2013,0,25,22154,999.0,1.0,241920.0,999.0,2.164806e+08,37,203284.0
3,01.2013,0,25,22154,999.0,1.0,241920.0,999.0,2.164806e+08,37,203284.0
4,01.2013,0,25,22154,999.0,1.0,241920.0,999.0,2.164806e+08,37,203284.0
...,...,...,...,...,...,...,...,...,...,...,...
2935844,10.2015,33,55,13093,250.0,1.0,63388.0,250.0,4.979206e+07,36,22.0
2935845,09.2015,32,55,13091,1000.0,1.0,63388.0,1000.0,4.979206e+07,36,22.0
2935846,09.2015,32,55,13094,2500.0,1.0,63388.0,2500.0,4.979206e+07,36,22.0
2935847,09.2015,32,55,13094,2500.0,2.0,63388.0,5000.0,4.979206e+07,36,22.0


In [43]:
train_groupby_item = train_groupby_item.rename(columns = {'item_cnt_day' : '#4_amount'}, inplace = False)

In [44]:
train = pd.merge(train, train_groupby_item , on='item_id')

In [45]:
train = train.drop(['item_cnt_day_x'], axis = 1)

In [46]:
train['month'] = train['date'].str[:2]

In [47]:
train['year'] = train['date'].str[3:]

In [48]:
def normalize_column(df, column_name):
    x = df[[column_name]]
    # Create x, where x the 'scores' column's values as floats
    x = x.values.astype(float)
    # Create a minimum and maximum processor object
    min_max_scaler = preprocessing.MinMaxScaler()
    # Create an object to transform the data to fit minmax processor
    x_scaled = min_max_scaler.fit_transform(x)
    # Run the normalizer on the dataframe
    df_normalized = pd.DataFrame(x_scaled, columns = [column_name])
    
    df[column_name] = df_normalized
    
    return df

In [49]:
normalize_column(train, '#2_amount')
normalize_column(train, '#2_price')
normalize_column(train, '#3_amount')
normalize_column(train, '#4_amount')
train['month'] = train['month'].astype(int)
train['year'] = train['year'].astype(int)

In [50]:
train.drop('tmp', axis = 1)

,date,date_block_num,shop_id,item_id,item_price,#2_amount,#2_price,item_category_id,#3_amount,#4_amount,month,year
0,01.2013,0,59,22154,999.0,0.156751,0.190977,37,0.320550,0.000373,1,2013
1,01.2013,0,25,22154,999.0,0.778200,0.920216,37,0.320550,0.000373,1,2013
2,01.2013,0,25,22154,999.0,0.778200,0.920216,37,0.320550,0.000373,1,2013
3,01.2013,0,25,22154,999.0,0.778200,0.920216,37,0.320550,0.000373,1,2013
4,01.2013,0,25,22154,999.0,0.778200,0.920216,37,0.320550,0.000373,1,2013
...,...,...,...,...,...,...,...,...,...,...,...,...
2935844,10.2015,33,55,13093,250.0,0.203120,0.210418,36,0.000033,0.000123,10,2015
2935845,09.2015,32,55,13091,1000.0,0.203120,0.210418,36,0.000033,0.000064,9,2015
2935846,09.2015,32,55,13094,2500.0,0.203120,0.210418,36,0.000033,0.000075,9,2015
2935847,09.2015,32,55,13094,2500.0,0.203120,0.210418,36,0.000033,0.000075,9,2015


# 6.1

In [59]:
train['item_price_avg'] = train['item_price'].mean()

1.0
0.0


# 6.2

In [247]:
out = []
unique_cat = train['item_category_id'].unique()
for cat in unique_cat:
    same_cat_diff_shop = train[train['item_category_id']==cat]
    same_shop = same_cat_diff_shop.drop_duplicates(subset='shop_id', keep="last") #unique shops
    norm=(same_shop['item_price']-same_shop['item_price'].mean())/same_shop['item_price'].std()
    same_shop['item_cat_price_norm'] = norm
    arr = np.array(same_shop)[:,[2,8, -1]]
    out.append(arr)
    #out.append(same_shop['date','item_category_id' ,'item_cat_price_norm'])
    #train = train.merge(same_shop, on = list(train.columns), how = 'left')

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pa

[[9 37 5.487175771473146]
 [59 37 -0.2000926380595216]
 [40 37 0.4689977630619688]
 [33 37 1.4726333647442043]
 [11 37 0.5559795152077626]
 [36 37 -0.5346378386202668]
 [25 37 1.4726333647442043]
 [49 37 1.4726333647442043]
 [24 37 -0.2000926380595216]
 [23 37 0.4689977630619688]
 [19 37 -0.48566042125817377]
 [22 37 -0.5335003849383603]
 [3 37 -0.20343809006512906]
 [39 37 -0.8725284911866195]
 [48 37 -0.2000926380595216]
 [18 37 -0.2000926380595216]
 [8 37 0.6362703633423413]
 [21 37 0.4689977630619688]
 [28 37 0.6362703633423413]
 [53 37 -0.2000926380595216]
 [27 37 -0.5346378386202668]
 [29 37 -0.7019104389006394]
 [43 37 0.6362703633423413]
 [26 37 0.4689977630619688]
 [4 37 0.8035429636227139]
 [6 37 0.6362703633423413]
 [2 37 0.6362703633423413]
 [7 37 -0.6556428376630883]
 [13 37 -1.1100555835847485]
 [0 37 -0.2536198701492408]
 [1 37 -1.1100555835847485]
 [16 37 -0.37071069034550164]
 [32 37 -0.7052558909062469]
 [56 37 -0.7052558909062469]
 [15 37 -0.6176385028793877]
 [58 37

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pa

[[11 19 0.9322216032021929]
 [40 19 0.9994722045207434]
 [36 19 -0.009286815257513221]
 [59 19 1.3357252111134956]
 [34 19 2.1763577275953763]
 [9 19 1.7686509571016642]
 [25 19 1.5038517144098718]
 [24 19 -0.7272290159588632]
 [23 19 -0.3463804543667473]
 [19 19 4.698255277041018]
 [13 19 0.006685202555642508]
 [22 19 -1.1617939953541714]
 [27 19 0.07477643639067483]
 [58 19 -0.5556979509707356]
 [17 19 0.07477643639067483]
 [49 19 -0.5552776347124946]
 [18 19 0.4950926946316151]
 [53 19 -0.7666967126076876]
 [35 19 -0.7658560800912056]
 [21 19 -0.7658560800912056]
 [28 19 -0.5140866414048825]
 [29 19 0.20045099760471596]
 [26 19 -0.9768548417281577]
 [54 19 0.32696619133523896]
 [4 19 -0.17741331855388934]
 [6 19 -1.0188864675522518]
 [2 19 0.6632191979279912]
 [48 19 0.6632191979279912]
 [3 19 -0.009286815257513221]
 [7 19 0.9154089528725553]
 [0 19 0.15715842300589913]
 [1 19 -0.7994813807504809]
 [16 19 -0.5552776347124946]
 [15 19 -0.8507599642558756]
 [8 19 -0.5552776347124946]


C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pa

[[1 29 -0.8899519768169163]
 [32 29 -1.1395741619776365]
 [13 29 -0.038453303394111224]
 [36 29 -0.4655072856725559]
 [33 29 0.3172800545526991]
 [11 29 0.09114148959873654]
 [30 29 -0.6829482135129045]
 [17 29 -0.6829482135129045]
 [40 29 -0.6829482135129045]
 [27 29 2.143783848411627]
 [29 29 2.143783848411627]
 [56 29 2.143783848411627]
 [51 29 1.6688928620083059]
 [59 29 0.8391382813695357]
 [19 29 0.8391382813695357]
 [18 29 0.8391382813695357]
 [28 29 0.8391382813695357]
 [4 29 0.7091086065210073]
 [2 29 0.8391382813695357]
 [3 29 0.8391382813695357]
 [16 29 0.8391382813695357]
 [53 29 0.3607682401207688]
 [35 29 0.8391382813695357]
 [52 29 0.8391382813695357]
 [41 29 0.8391382813695357]
 [46 29 0.8391382813695357]
 [45 29 0.8391382813695357]
 [5 29 0.7091086065210073]
 [58 29 0.8391382813695357]
 [9 29 0.6216973535291871]
 [49 29 0.8391382813695357]
 [39 29 0.8391382813695357]
 [48 29 0.8391382813695357]
 [34 29 0.8391382813695357]
 [7 29 -0.7699245846490439]
 [37 29 -0.76992458

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pa

[[23 3 -0.8954002017651285]
 [0 3 -1.0778452821466977]
 [1 3 -1.0778452821466977]
 [32 3 -0.8954002017651285]
 [40 3 1.0149488694675681]
 [13 3 0.30078641474438245]
 [11 3 -0.8967774136559465]
 [9 3 -0.5404486835130481]
 [33 3 -0.3338668998903373]
 [34 3 -1.0728759608911687]
 [36 3 -1.0728759608911687]
 [59 3 1.8668325132725612]
 [17 3 -0.6824292908138802]
 [29 3 -0.6824292908138802]
 [10 3 -0.8925605896191118]
 [30 3 -0.6824292908138802]
 [51 3 -0.8925605896191118]
 [27 3 0.16306522566257525]
 [4 3 -0.2628765962399212]
 [54 3 0.16306522566257525]
 [43 3 0.16306522566257525]
 [38 3 -0.2628765962399212]
 [58 3 -0.2628765962399212]
 [49 3 -0.2628765962399212]
 [48 3 1.795842209622145]
 [50 3 2.7897064607279702]
 [45 3 2.7897064607279702]
 [5 3 2.7897064607279702]
 [19 3 -0.7534195944642963]
 [22 3 -0.7534195944642963]
 [18 3 -0.7534195944642963]
 [7 3 -0.7534195944642963]
 [15 3 -0.7534195944642963]
 [44 3 -0.7534195944642963]
 [37 3 -0.7534195944642963]
 [39 3 -0.8528060195748788]
 [25 

[[12 68 -0.8018790486015399]
 [25 68 1.1204827777334019]
 [31 68 -0.3186037291318618]]
[[25 82 -0.11779195769754906]
 [22 82 -0.11779195769754906]
 [18 82 -0.11779195769754906]
 [29 82 -0.11779195769754906]
 [6 82 -0.11779195769754906]
 [1 82 -1.27888411214482]
 [16 82 -0.11779195769754906]
 [13 82 -1.27888411214482]
 [12 82 -1.27888411214482]
 [30 82 -0.11779195769754906]
 [56 82 -0.11779195769754906]
 [52 82 -0.11779195769754906]
 [51 82 -1.27888411214482]
 [41 82 -0.11779195769754906]
 [37 82 -0.11779195769754906]
 [57 82 -0.11779195769754906]
 [58 82 -0.11779195769754906]
 [2 82 1.8173616330479025]
 [0 82 -0.5048226758466394]
 [4 82 2.2043923511969927]
 [10 82 -0.5048226758466394]
 [54 82 1.8173616330479025]
 [44 82 1.8173616330479025]]
[[4 81 -1.7163438342810018]
 [25 81 0.9870740145712836]
 [22 81 0.9870740145712836]
 [29 81 0.9870740145712836]
 [0 81 -0.6049387186417289]
 [1 81 -0.6049387186417289]
 [16 81 0.9870740145712836]
 [12 81 -0.508817195126981]
 [30 81 0.987074014571283

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pa

In [253]:
out = np.array(out)
v = np.vstack(out)
df = pd.DataFrame(v, columns = ['shop_id', 'item_category_id', 'item_cat_price_norm'])
df = df.fillna(0)
df_train = train.merge(df, on = ['shop_id', 'item_category_id'], how = 'left')
train = df_train.drop(['item_cat_price_norm_x'], axis = 1)

[[55 36 nan]]


In [254]:
train

,date,date_block_num,shop_id,item_id,item_price,#2_amount,tmp,#2_price,item_category_id,#3_amount,#4_amount,month,year,item_price_avg,item_cat_price_norm_y
0,01.2013,0,59,22154,999.0,0.156751,999.0,0.190977,37,0.320550,0.000373,1,2013,0.003247,-0.200093
1,01.2013,0,25,22154,999.0,0.778200,999.0,0.920216,37,0.320550,0.000373,1,2013,0.003247,1.472633
2,01.2013,0,25,22154,999.0,0.778200,999.0,0.920216,37,0.320550,0.000373,1,2013,0.003247,1.472633
3,01.2013,0,25,22154,999.0,0.778200,999.0,0.920216,37,0.320550,0.000373,1,2013,0.003247,1.472633
4,01.2013,0,25,22154,999.0,0.778200,999.0,0.920216,37,0.320550,0.000373,1,2013,0.003247,1.472633
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2935844,10.2015,33,55,13093,250.0,0.203120,250.0,0.210418,36,0.000033,0.000123,10,2015,0.000815,0.000000
2935845,09.2015,32,55,13091,1000.0,0.203120,1000.0,0.210418,36,0.000033,0.000064,9,2015,0.003250,0.000000
2935846,09.2015,32,55,13094,2500.0,0.203120,2500.0,0.210418,36,0.000033,0.000075,9,2015,0.008121,0.000000
2935847,09.2015,32,55,13094,2500.0,0.203120,5000.0,0.210418,36,0.000033,0.000075,9,2015,0.008121,0.000000


# 1

In [258]:
train = pd.concat([train,pd.get_dummies(train['shop_id'], prefix='shop_id')],axis=1)

In [259]:
train

,date,date_block_num,shop_id,item_id,item_price,#2_amount,tmp,#2_price,item_category_id,#3_amount,...,shop_id_50,shop_id_51,shop_id_52,shop_id_53,shop_id_54,shop_id_55,shop_id_56,shop_id_57,shop_id_58,shop_id_59
0,01.2013,0,59,22154,999.0,0.156751,999.0,0.190977,37,0.320550,...,0,0,0,0,0,0,0,0,0,1
1,01.2013,0,25,22154,999.0,0.778200,999.0,0.920216,37,0.320550,...,0,0,0,0,0,0,0,0,0,0
2,01.2013,0,25,22154,999.0,0.778200,999.0,0.920216,37,0.320550,...,0,0,0,0,0,0,0,0,0,0
3,01.2013,0,25,22154,999.0,0.778200,999.0,0.920216,37,0.320550,...,0,0,0,0,0,0,0,0,0,0
4,01.2013,0,25,22154,999.0,0.778200,999.0,0.920216,37,0.320550,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2935844,10.2015,33,55,13093,250.0,0.203120,250.0,0.210418,36,0.000033,...,0,0,0,0,0,1,0,0,0,0
2935845,09.2015,32,55,13091,1000.0,0.203120,1000.0,0.210418,36,0.000033,...,0,0,0,0,0,1,0,0,0,0
2935846,09.2015,32,55,13094,2500.0,0.203120,2500.0,0.210418,36,0.000033,...,0,0,0,0,0,1,0,0,0,0
2935847,09.2015,32,55,13094,2500.0,0.203120,5000.0,0.210418,36,0.000033,...,0,0,0,0,0,1,0,0,0,0
